In [1]:
import pandas as pd
info_filename1 = r"E:\pycharm files\housing_pricing_model\data\train_data.csv"
info_filename2 = r"E:\pycharm files\housing_pricing_model\data\jw.csv"
df_data = pd.read_csv(info_filename1,encoding="gbk")
df_jw = pd.read_csv(info_filename2,encoding="utf-8")
df_data = df_data.iloc[:,1:53]
df = pd.concat([df_data,df_jw],axis=1)


In [2]:
i = len(df.columns)
df_X = df.iloc[:,1:i]
df_y = df.iloc[:,[0]]

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def scalerXY(df_X,df_y):
    scalerX = StandardScaler()
    scalery = MinMaxScaler()
    
    df_scaledX = pd.DataFrame(scalerX.fit_transform(df_X), columns=df_X.columns)
    df_scaledy = pd.DataFrame(scalery.fit_transform(df_y), columns=df_y.columns)
    
    df_scaledy = df_scaledy *0.98+0.01
    df_scaledX = df_scaledX*0.98+0.01
    
    return (scalerX,scalery,df_scaledX,df_scaledy)

def scalerX(df_X,scaler_cols):
    scalerX = StandardScaler()
    idx = df_X.index
    # 需要标准化的列
    df_scaler_col = df_X[scaler_cols]
    
    cols = np.array(df_X.columns)
    scaler_cols = np.array(scaler_cols)
    keep_cols = np.setdiff1d(cols,scaler_cols)
    # 不需要标准化的列
    df_keep_col = df_X[keep_cols]
    
    df_scaler_col = pd.DataFrame(scalerX.fit_transform(df_scaler_col), index=idx,columns=df_scaler_col.columns)
    
    df_scaler = pd.concat([df_scaler_col,df_keep_col],axis=1)
    df_scaler = pd.DataFrame(df_scaler,columns=cols)
    df_scaler = df_scaler*0.98+0.01
    
    # print(len(df_X))
    # print(len(df_keep_col))
    # print(len(df_scaler_col))
    # print(len(df_scaler))
    return (scalerX,df_scaler)

def scalerX_inverse(scalerX,df_scaler,scaler_cols):
    idx = df_scaler.index
    # 经过标准化的列
    df_scaler_col = df_scaler[scaler_cols]
    
    cols = np.array(df_scaler.columns)
    scaler_cols = np.array(scaler_cols)
    keep_cols = np.setdiff1d(cols,scaler_cols)
    # 没有经过标准化的列
    df_keep_col = df_scaler[keep_cols]
    
    

def scalerY(df_y):
    scalery = MinMaxScaler()
    df_scaledy = pd.DataFrame(scalery.fit_transform(df_y), columns=df_y.columns)
    df_scaledy = df_scaledy *0.98+0.01
    return (scalery,df_scaledy)


In [4]:

# scalerX_train,df_scaler = scalerX(df_X,scaler_cols)

In [5]:
from sklearn.model_selection import train_test_split
def tt_split(X,y,test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=test_size)
    return (X_train, X_test, y_train, y_test)

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.style as style
style.use('ggplot')

def drawhist(data):
    
    # data = df_X['挂牌时间']
    
    # 绘制概率密度图
    plt.hist(data, bins=30, density=True, alpha=0.5, color='blue')
    
    # 添加标题和标签
    # plt.title("Probability Density Histogram")
    plt.xlabel("Value")
    plt.ylabel("Probability Density")
    
    # 显示图形
    plt.show()
    
# for i in df_X.columns:
#     data = df_X[i]
#     drawhist(data)
#     plt.title(str(i))
#     

In [7]:
import 机器学习.util.ANN as ANN

def ANNtrain(X_train,y_train,learning_rate,batch_size,epochs):
    input_lay = X_train.shape[1]
    output_lay = y_train.shape[1]
    print(input_lay,output_lay)
    ann = ANN.ANN(layer_number=4,
                  layer_neuron=[input_lay,120,110,output_lay],
                  learning_rate=learning_rate,
                  batch_size=batch_size,
                  epochs=epochs)
    
    ann_param = ann.fit(X_train,y_train)
    print(f"loss={ann_param[2]}")
    return (ann,ann_param)


In [8]:

import numpy as np
from sklearn.neighbors import NearestNeighbors


def NN(k,X_train,X_test,y_train,y_test):
    # 使用NearestNeighbors找到每个X_test样本的K个最近邻
    # k = 4  # 选择K的值
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(X_train)
    distances, indices = nbrs.kneighbors(X_test)
    
    y_predict = np.zeros((len(y_test),1))
    # 打印每个X_test样本的K个最近邻的索引和距离
    for i in range(len(y_test)):
        idx = indices[i]
        # print(f"样本{i}的K个最近邻索引：", indices[i])
        # print(f"样本{i}的K个最近邻距离：", distances[i])
        # print(y_train[idx])
        # print(y_train[idx].mean())
        # print(y_test[i])
        # print(idx)
        y_predict[i,0] = y_train.iloc[idx,:].mean()
    return y_predict,nbrs
    
def nbrs_test(nbrs,X_train,X_test,y_train,y_test):
    
    y_predict = np.zeros((len(y_test),1))
    distances, indices = nbrs.kneighbors(X_test)
    for i in range(len(y_test)):
        idx = indices[i]
        # print(f"样本{i}的K个最近邻索引：", indices[i])
        # print(f"样本{i}的K个最近邻距离：", distances[i])
        # print(y_train[idx])
        # print(y_train[idx].mean())
        # print(y_test[i])
        # print(idx)
        y_predict[i,0] = y_train.iloc[idx,:].mean()
    
    return y_predict

In [9]:
df_X_train, df_X_test, df_y_train, df_y_test= tt_split(df_X,df_y,0.2)

In [10]:
y_predict,nbrs_train = NN\
    (5,df_X_train.loc[:,["经度",'纬度']],df_X_train.loc[:,["经度",'纬度']],df_y_train,df_y_train)

# y_error = abs(y_predict-df_y_train.to_numpy().reshape(-1,1))
# idx_error = y_error>2000
# print()
# print(f'错误率：{idx_error.sum()/len(df_y_train)}')

df_knnjw = pd.DataFrame(y_predict,columns=['knnjw'],index=df_X_train.index)
df_X_train = pd.concat([df_X_train,df_knnjw],axis=1)
# df_X_train.drop(["经度","纬度"],inplace=True,axis=1)
# df_X_train.drop_duplicates(inplace=True)

In [11]:

y_predict,nbrs_test = NN(5,df_X_train.loc[:,["经度",'纬度']],df_X_test.loc[:,["经度",'纬度']],df_y_train,df_y_test)
df_knnjw_test = pd.DataFrame(y_predict,columns=['knnjw'],index=df_X_test.index)
df_X_test = pd.concat([df_X_test,df_knnjw_test],axis=1)
# df_X_test.drop(["经度","纬度"],inplace=True,axis=1)
# df_X_train.drop_duplicates(inplace=True)

In [12]:
# df_X_test.drop(["经度","纬度"],inplace=True,axis=1)
# df_X_train.drop(["经度","纬度"],inplace=True,axis=1)

In [13]:
# 标准化
# scalerX_train,scalery_train,df_scaledX_train,df_scaledy_train\
#     = scalerXY(df_X_train,df_y_train)
# scalerX_test,scalery_test,df_scaledX_test,df_scaledy_test\
#     = scalerXY(df_X_test,df_y_test)
scaler_cols = ['建筑面积', '总楼层', '挂牌时间', '物业费用', '建筑年代', '绿  化  率', '容  积  率',
        '总楼栋数', '总  户  数', '产权年限', '卧室', '客厅', '卫生间','经度','纬度','knnjw']
scalerX_train,df_scaledX_train = scalerX(df_X_train,scaler_cols)
scalery_train,df_scaledy_train = scalerY(df_y_train)

scalerX_test,df_scaledX_test = scalerX(df_X_test,scaler_cols)
scalery_test,df_scaledy_test = scalerY(df_y_test)

In [14]:
print(len(df_X_train))

2380


In [15]:
X_train = df_scaledX_train.to_numpy()
y_train = df_scaledy_train.to_numpy().reshape(-1,1)

X_test = df_scaledX_test.to_numpy()
y_test = df_scaledy_test.to_numpy().reshape(-1,1)

In [16]:
print(len(df_scaledX_train))

2380


In [39]:
ann,ann_param = ANNtrain(X_train,y_train,0.17,5,1200)

54 1
第0次epoch loss=0.044404752362901305
第100次epoch loss=0.04440475184325846
第200次epoch loss=0.0444047512479687
第300次epoch loss=0.04440475055914594
第400次epoch loss=0.044404749752770215


KeyboardInterrupt: 

In [36]:
y_hat = ann.transform(X_test)


In [34]:
import pickle
file_name = r"E:\pycharm files\housing_pricing_model\data\ann_list.bin"
    # 从二进制文件中读取列表
with open(file_name, 'rb') as f:
    loaded_list = pickle.load(f)
ann1 = loaded_list[0]
y_hat = ann1.transform(X_test)

In [37]:
def pred_error(y_pred,y_true,scalery_test,abse):
    y_true = (y_true-0.01)/0.98
    y_pred = (y_pred-0.01)/0.98
    df_y_true = scalery_test.inverse_transform(y_true)
    df_y_pred = scalery_test.inverse_transform(y_pred)
    y_error = df_y_true - df_y_pred
    idx_error = abs(y_error)>abse
    # print(len(y_true))
    print(idx_error.sum())    
    acc = idx_error.sum()/len(y_true)
    print(f"错误率：{acc}")
    return acc,idx_error

acc2000_ann,idx_error2000_ann = pred_error(y_hat,y_test,scalery_test,2000)
acc3000_ann,idx_error3000_ann = pred_error(y_hat,y_test,scalery_test,3000)
acc4000_ann,idx_error4000_ann = pred_error(y_hat,y_test,scalery_test,4000)

324
错误率：0.5445378151260504
196
错误率：0.32941176470588235
116
错误率：0.1949579831932773


In [20]:
import pickle

# 要持久化的列表
def save_ann_param(ann,ann_param,acc):
    ann_list = [ann,ann_param,acc]
    
    # 将列表持久化为二进制文件
    file_name = r"E:\pycharm files\housing_pricing_model\data\ann_list.bin"
    with open(file_name, 'wb') as f:
        pickle.dump(ann_list, f)
    print("save success")
    # 从二进制文件中读取列表
    # with open(file_name, 'rb') as f:
    #     loaded_list = pickle.load(f)
    
    # 输出读取到的列表
    # print(loaded_list)
# save_ann_param(ann,ann_param,(acc2000_ann,acc3000_ann,acc4000_ann))

In [88]:

# df_scaledX_test
# df_scaledX_test = (df_scaledX_test-0.01)/0.98
# scalerX_test.inverse_transform(df_scaledX_test)

In [30]:
# for i in df_X.columns:
#     data = df_error_test[i]
#     drawhist(data)
#     # plt.title(str(i))

In [118]:
# 决策树
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# 创建一个示例数据集

# 创建决策树回归模型
model = DecisionTreeRegressor()

# 拟合模型
model.fit(X_train, y_train)

# 对新数据进行np.array([[6]])
y_pred = model.predict(X_test)
# 
# # 输出预测结果

y_pred = y_pred.reshape(-1,1)
pred_error(y_pred,y_test,scalery_test,2000)
pred_error(y_pred,y_test,scalery_test,3000)
pred_error(y_pred,y_test,scalery_test,4000)
print()

278
错误率：0.4672268907563025
182
错误率：0.3058823529411765
123
错误率：0.20672268907563024



In [33]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np


# 创建随机森林回归模型
model = RandomForestRegressor(n_estimators=50)

# 拟合模型
model.fit(X_train, y_train)

# 对新数据进行预测

y_pred = model.predict(X_test)

# 输出预测结果
y_pred = y_pred.reshape(-1,1)
acc2000_forest,idx_error2000_forest = pred_error(y_pred,y_test,scalery_test,2000)
acc3000_forest,idx_error3000_forest = pred_error(y_pred,y_test,scalery_test,3000)
acc4000_forest,idx_error4000_forest = pred_error(y_pred,y_test,scalery_test,4000)

C:\Users\clarifyC\AppData\Local\Temp/ipykernel_12204/30503895.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


338
错误率：0.5680672268907563
221
错误率：0.37142857142857144
129
错误率：0.21680672268907564


In [28]:
idx_error2000 = idx_error2000_forest & idx_error2000_ann

print(idx_error2000.sum())

257


In [25]:
# AdaBoost（Adaptive Boosting）：
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(n_estimators=100)

# 拟合模型
model.fit(X_train, y_train)

# 预测
y_pred = model.predict(X_test)
# 输出预测结果
y_pred = y_pred.reshape(-1,1)
pred_error(y_pred,y_test,scalery_test,2000)
pred_error(y_pred,y_test,scalery_test,3000)
pred_error(y_pred,y_test,scalery_test,4000)

D:\Anaconda3_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


453
错误率：0.761344537815126
365
错误率：0.6134453781512605
284
错误率：0.4773109243697479


(0.4773109243697479,
 array([[ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [ 

In [84]:
# 梯度提升树（Gradient Boosting）
from sklearn.ensemble import GradientBoostingRegressor

# 创建梯度提升树分类器
model = GradientBoostingRegressor(n_estimators=100)

# 拟合模型
model.fit(X_train, y_train)

# 预测
y_pred = model.predict(X_test)


y_pred = y_pred.reshape(-1,1)
pred_error(y_pred,y_test,scalery_test,2000)
pred_error(y_pred,y_test,scalery_test,3000)
pred_error(y_pred,y_test,scalery_test,4000)

D:\Anaconda3_2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


252
错误率：0.4235294117647059
168
错误率：0.2823529411764706
112
错误率：0.18823529411764706
